# IBM 1 and 2, Expectation Maximization

f is the source language, e the target language. In this code, we use the convention that we represent our data in the following way:
(source, target, alignment)

### 1. First we collect all imports, read in the data and initialize the system's parameters

In [5]:
from collections import Counter, defaultdict
from aer import read_naacl_alignments, AERSufficientStatistics


import random
import codecs
import math
import tqdm
import pprint
import numpy as np

# Number of iterations
S = 10

# Set paths for data
training_english_path = "training/hansards.36.2.e"
training_french_path = "training/hansards.36.2.f"
validation_english_path = "validation/dev.e"
validation_french_path = "validation/dev.f"

### 2. Read in the data

In [2]:
with codecs.open(training_english_path, 'r', 'utf8') as f:
    training_english = [line.split() for line in f.readlines()]

with codecs.open(training_french_path, 'r', 'utf8') as f:
    training_french = [line.split() for line in f.readlines()]

training_data = list(zip(training_french, training_english))

# Add NULL characters at the start of english sentences
for i, (f, e) in enumerate(training_data):
    e = ["NULL"] + e
    training_data[i] = (f, e)

with codecs.open(validation_english_path, 'r', 'utf8') as f:
    validation_english = [line.split() for line in f.readlines()]

with codecs.open(validation_french_path, 'r', 'utf8') as f:
    validation_french = [line.split() for line in f.readlines()]

validation_data = list(zip(validation_french, validation_english))

### 3. The IBM1 model EM

In [10]:
class IBM1_EM:
    def __init__(self, training_data, validation_data, valid_align_path, training_iterations):
        self.data = training_data
        self.validation_data = validation_data
        self.training_iterations = training_iterations
        self.valid_align_path = valid_align_path
        self.t = self.init_t()

    def train(self):
        """Train IBM1 by expectation maximization"""
        for s in range(0, self.training_iterations):
            ll = self.log_likelihood()
            print("Log likelihood: {}".format(ll))
            self.test()
            print("Iteration {}".format(s + 1))
            c_1 = Counter()
            c_2 = Counter()
            n = len(self.data)

            for k in tqdm.tqdm(range(n)):
                # extract all info for the current sentence 
                pair = self.data[k]
                e_sentence = pair[1]
                f_sentence = pair[0]

                # loop over all positions in both sentences
                for f in f_sentence:
                    sentence_prob = sum([self.t[(f, e2)] for e2 in e_sentence])
                    for e in e_sentence:
                        delta = self.t[(f, e)] / sentence_prob
                        # update the counts
                        c_1[(e, f)] += delta
                        c_2[e] += delta

            # after looping over the counts, re-estimate t and q
            self.update_t(c_1, c_2)

    def init_t(self):
        """Initialize the transition probabilities randomly. This is a counter object."""
        vocabulary = defaultdict(list)
        for f, e in self.data:
            for w1 in f:
                for w2 in e:
                    vocabulary[w2].append(w1)
        t = Counter()
        for e in vocabulary:
            words = list(set(vocabulary[e]))
            probs = np.array([1 for i in range(len(words))])
            probs = probs / sum(probs)
            for i, f in enumerate(words):
                t[(f, e)] = probs[i] 
        return t

    def update_t(self, c_1, c_2):
        """Update the transition probabilities.

        Args:
            c_1: counts for english and french words occurring together
            c_2: counts for english words on their own

        Returns:
            Counter object
        """
        for all_f, all_e in self.data:
            for f in all_f:
                for e in all_e:
                    self.t[(f, e)] = c_1[(e, f)] / c_2[e]

    def log_likelihood(self):
        """Calculate log likelihood of IBM1 model.

        Args:
            data: list of aligned sentences in tuples (french, english)
            t: Counter object, transition probabilities

        Returns:
            float
        """
        log_likelihood = 0
        for all_f, all_e in self.data:
            likelihood = 1
            # Sum over all alignments using ibm1 trick
            for f in all_f:
                probs = []
                for e in all_e:
                    probs.append(self.t[(f, e)])
                likelihood *= sum(probs)
            likelihood = ((1 / float(1 + len(all_e)))**(len(all_f))) * likelihood
            if likelihood != 0:
                log_likelihood += math.log(likelihood)
        return log_likelihood

    def align(self, f_sentence, e_sentence):
        alignment = []
        for i, f in enumerate(f_sentence):
            alignment_i = None
            best_score = -1
            for j, e in enumerate(e_sentence):
                score = self.t[(f, e)]
                if score > best_score:
                    best_score = score
                    alignment_i = j
            alignment.append((alignment_i + 1, i + 1))
        return alignment

    def test(self):
        from random import random
        # 1. Read in gold alignments
        gold_sets = read_naacl_alignments(self.valid_align_path)

        # 2. Here you would have the predictions of your own algorithm, 
        #  for the sake of the illustration, I will cheat and make some predictions by corrupting 50% of sure gold alignments
        predictions = []
        for i, (f, e) in enumerate(self.validation_data):
            links = set(self.align(f, e))
            predictions.append(links)

        # 3. Compute AER

        # first we get an object that manages sufficient statistics 
        metric = AERSufficientStatistics()
        # then we iterate over the corpus 
        for gold, pred in zip(gold_sets, predictions):
            metric.update(sure=gold[0], probable=gold[1], predicted=pred)
        # AER
        print(metric.aer())


### 4. Train our IBM1 model

In [ ]:
model = IBM1_EM(training_data, validation_data, 'validation/dev.wa.nonullalign', 10)
model.train()

Log likelihood: -35225718.83664254
0.8309726156751652
Iteration 1



100%|█████████████████████████████████| 231164/231164 [08:11<00:00, 470.77it/s]


In [214]:
# Check whether the data makes sense
pprint.pprint(model.t.most_common(25))

[(('prière', 'prayers'), 1.0),
 (('*', '*'), 0.90384884150794809),
 (('ensemble', 'Together'), 0.5),
 (('PLÉNIERS', 'WHOLE'), 0.5),
 (('travailler', 'Together'), 0.5),
 (('COMITÉS', 'WHOLE'), 0.5),
 (('le', 'Infrastructure'), 0.34144480421226403),
 (('YORK', 'WEST'), 0.33333333333333337),
 (('YORK', 'YORK'), 0.33333333333333337),
 (('-', 'WEST'), 0.33333333333333337),
 (('OUEST', 'YORK'), 0.33333333333333337),
 (('-', 'YORK'), 0.33333333333333337),
 (('OUEST', 'WEST'), 0.33333333333333337),
 (('chambre', 'COMMONS'), 0.33333333333333331),
 (('-', 'ROYAL'), 0.33333333333333331),
 (('COMMUNES', 'COMMONS'), 0.33333333333333331),
 (('ROYAL', 'ROYAL'), 0.33333333333333331),
 (('chambre', 'house'), 0.33333333333333331),
 (('mont', 'Royal'), 0.33333333333333331),
 (('table', 'contents'), 0.33333333333333331),
 (('MATIÈRES', 'contents'), 0.33333333333333331),
 (('DES', 'contents'), 0.33333333333333331),
 (('DES', 'house'), 0.33333333333333331),
 (('SIÈGE', 'vacancies'), 0.33333333333333331),
 (

0.5561850802644004
